In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pandas as pd

import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import seaborn as sns
from IPython.display import clear_output

import random


# from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re

# from torch import optim
# import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [3]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=28910d71e6a363d268bde8375432e70f29f116912ad16a2d7b45264112eaa9f7
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [4]:
import wandb

In [5]:
wandb.login(key='6264e29f109d3cf02a9911cd18b6725b79c489cd')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
!pip install gdown
import gdown 
# !conda install -y gdown
import zipfile
url = 'https://drive.google.com/u/0/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw&export=download'  
gdown.download(url)
z= zipfile.ZipFile('aksharantar_sampled.zip')
z.extractall()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/u/0/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw&export=download
To: /content/aksharantar_sampled.zip
100%|██████████| 14.0M/14.0M [00:00<00:00, 231MB/s]


In [7]:
df = pd.read_csv("aksharantar_sampled/tel/tel_train.csv" , header = None)
# df.head()
df_val = pd.read_csv("aksharantar_sampled/tel/tel_valid.csv" , header = None)
df_test = pd.read_csv("aksharantar_sampled/tel/tel_test.csv" , header = None)
x1 = 0
english_words_val = df_val[x1]
english_words_test = df_test[x1]
english_words = df[x1]
telugu_words_val = df_val[x1+1]
telugu_words_test = df_test[x1+1]
telugu_words = df[x1+1]
#print(df[50000:])

In [8]:
def findCharList(eng_words,tel_words):
  english_char_list = []
  max_length_word_english = -1
  telugu_char_list = []
  max_length_word_telugu = -1
  for word in eng_words:
    x2 = len(word)
    x3 = max(max_length_word_english,x2)
    max_length_word_english = x3
    for char in word :
      c = char
      english_char_list.append(c);
  english_char_list = set(english_char_list)
  english_char_list = list(english_char_list)
  for word in tel_words:
    x4 = len(word)
    x5 = max(max_length_word_telugu,x4)
    max_length_word_telugu = x5
    for char in word :
      c1 =char
      telugu_char_list.append(c1);
  telugu_char_list = set(telugu_char_list)
  telugu_char_list = list(telugu_char_list)
  english_char_list.sort()
  telugu_char_list.sort()
  # print(len(telugu_char_list))
  #print(telugu_char_list)
  print(len(telugu_char_list))
  return english_char_list,telugu_char_list,max_length_word_english,max_length_word_telugu

In [9]:
english_char_list,telugu_char_list,max_length_word_english,max_length_word_telugu=findCharList(english_words,telugu_words)
print(english_char_list)
print(telugu_char_list)
print(max_length_word_telugu)
print(max_length_word_english)

62
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['ం', 'ః', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'వ', 'శ', 'ష', 'స', 'హ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', '్']
21
28


In [10]:
# for one word.
def english_word2vec(word):
  vec = []
  x6 = len(english_char_list)+1
  vec.append(x6)
  for char in word:
    c = char
    x7 = len(english_char_list)
    for i in range(x7):
      x8 = english_char_list[i]
      if(x8 == c):
        vec.append(i+1)
  x9 = max_length_word_english + 1
  while(len(vec) < x9): 
      x10 = 0
      vec.append(x10)
  x11 = 0
  vec.append(x11)
  return (vec)

In [11]:
def telugu_word2vec(word):
  vec = []
  x6 = len(telugu_char_list)+1
  vec.append(x6)
  for char in word:
    c = char
    x7 = len(telugu_char_list)
    for i in range(x7):
      x8 = telugu_char_list[i]
      if(x8 == c):
        vec.append(i+1)
  x9 = max_length_word_telugu + 1
  while(len(vec) < x9):  
      x10 = 0
      vec.append(x10)
  x11 = 0
  vec.append(x11)
  return(vec)

In [12]:
pred = []
gt=[]
inp = []

In [13]:
# creating matrix of representation for whole words of english and telugu.

def ip_matrix_construct(words, lang):
  ans = []
  if(lang == "english"):
    for word in words:
      ans.append(english_word2vec(word))
  else:
    for word in words:
      ans.append(telugu_word2vec(word))
  return(ans)

In [14]:
def matrixRep(english_words,telugu_words,english_words_val,telugu_words_val,english_words_test,telugu_words_test):
  # calculated representations of whole english and marathi words in variables english and marathi matrix.
  eng = "english"
  english_matrix = ip_matrix_construct(english_words, eng)
  
  english_matrix_val = ip_matrix_construct(english_words_val, eng)
  english_matrix_test = ip_matrix_construct(english_words_test, eng)
  english_matrix = torch.tensor(english_matrix)
  english_matrix_val = torch.tensor(english_matrix_val)
  english_matrix_test = torch.tensor(english_matrix_test)
  
  tel = "telugu"
  telugu_matrix = ip_matrix_construct(telugu_words, tel)
 
  telugu_matrix_val = ip_matrix_construct(telugu_words_val, tel)
  telugu_matrix_test =ip_matrix_construct(telugu_words_test, tel)

  telugu_matrix = torch.tensor(telugu_matrix) 
  telugu_matrix_val = torch.tensor(telugu_matrix_val)
  telugu_matrix_test = torch.tensor(telugu_matrix_test)
  
  return english_matrix,telugu_matrix,english_matrix_val,telugu_matrix_val,english_matrix_test,telugu_matrix_test

In [15]:
english_matrix,telugu_matrix,english_matrix_val,telugu_matrix_val,english_matrix_test,telugu_matrix_test = matrixRep(english_words,telugu_words,english_words_val,telugu_words_val,english_words_test,telugu_words_test)

In [16]:
class Encoder(nn.Module):
  def __init__(self,input_size, embedding_size, hidden_size, num_layers, dropout, cell_type, bidirectional,s=0.1):
    super(Encoder,self).__init__()
    ct = cell_type
    self.cell_type = ct
    dp = dropout
    self.dropout = nn.Dropout(dp)
    bd = bidirectional
    self.bidirectional = bd
    hs = hidden_size
    self.hidden_size = hs
    iS = input_size 
    self.embedding = nn.Embedding(iS, embedding_size)
    nl = num_layers
    self.num_layers = nl
    s1 = s
    self.s = s1
    if(ct == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = dropout, bidirectional = bidirectional)
    elif(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout = dropout, bidirectional = bidirectional)
    elif(ct == "GRU"):
      s1 = s1*2
      self.gru = nn.GRU(embedding_size, hidden_size, num_layers, dropout = dropout, bidirectional = bidirectional)

  def forward(self, x):
    k = x
    embedding = self.dropout(self.embedding(k))
    ct1 = self.cell_type
    if(ct1 == "LSTM"):
      outputs, (hidden,cell) = self.lstm(embedding)
      return outputs,(hidden, cell)
    elif(ct1 == "GRU"):
      output, hidden = self.gru(embedding)
      return output, hidden
    elif(ct1 == "RNN"):
      output, hidden = self.rnn(embedding)
      return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
#Attention Decoder
class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout, cell_type,bidirectional,s=0.1):
    super(Decoder, self).__init__()
    self.cell_type = cell_type
    hs = hidden_size
    self.hidden_size = hs
    dp = dropout
    self.dropout = nn.Dropout(dp)
    iS = input_size
    self.bidirectional=bidirectional
    self.embedding = nn.Embedding(iS, embedding_size)
    self.s = s
    nl = num_layers
    self.num_layers = nl
    if(cell_type == "GRU"):
      self.gru = nn.GRU(hs, hs, nl, dropout = dropout)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(hs, hs, nl, dropout = dropout)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(hs, hs, nl, dropout = dropout)
    max_len_word=max_length_word_english+2
    self.attn=nn.Linear(hidden_size +embedding_size, max_len_word)
    
    if (bidirectional==True):
        self.attn_combine = nn.Linear(hs*2+embedding_size, hs)
    else:
        self.attn_combine = nn.Linear(hs +embedding_size, hs)
        
    self.fc = nn.Linear(hs, output_size)  
  
  def forward(self, x, hidden, cell = 0,enco_out = None):
    
    x = x.unsqueeze(0).int()

    embedding = self.dropout(self.embedding(x))
    emb=embedding[0]
    hid=hidden[0]
    atten_weights = F.softmax(self.attn(torch.cat((emb, hid), 1)), dim=1)
    atten_applied = torch.bmm(atten_weights.unsqueeze(1),enco_out.permute(1,0,2)).squeeze(1)
    att=atten_applied
    output = torch.cat((embedding[0], att), 1)

    output = F.relu(self.attn_combine(output).unsqueeze(0))
    ct=self.cell_type
    if(ct == "LSTM"):
        outputs, (hidden, cell) = self.lstm(output, (hidden, cell))
    elif(ct == "RNN"):
        outputs, hidden = self.rnn(output, hidden)
    elif(ct == "GRU"):
        outputs, hidden = self.gru(output, hidden)  
    
    predictions = self.fc(outputs)
    
    predictions = predictions.squeeze(0)
    if(ct == "LSTM"):
        return predictions, hidden, cell
    else:
        return predictions, hidden


  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type, bidirectional, num_layers,s=0.1):
        super(Seq2Seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        nl = num_layers
        self.num_layers = nl
        bd = bidirectional
        self.bidirectional = bd
        ct = cell_type
        self.s = s
        self.cell_type = ct
        

    def forward(self, source, target, teacher_force_ratio=0.5,p=0.1):
        t2 = target.shape[0]
        target_len = t2
        s2 = source.shape[1]
        batch_size = s2
        tcl = len(telugu_char_list)
        target_vocab_size = tcl + 2  # intitially = 66 || Now 63 + 1(start token) + 1 = 65
        tvs = target_vocab_size
        outputs = torch.zeros(t2, s2, tvs).to(device)
        ct3 = self.cell_type
        if(ct3 == "GRU"):
            enco_output, hidden = self.encoder(source)
        elif(ct3 == "RNN"):
            enco_output, hidden = self.encoder(source)
        else:
            enco_output,(hidden, cell) = self.encoder(source)
        bd3 = self.bidirectional 
        sl3 = self.num_layers
        if(bd3 == True):
            hidden = hidden[sl3 - 1] + hidden[sl3 - 1]
            hidden = hidden.repeat(sl3,1,1)
            if(self.cell_type == "LSTM"):
                cell = cell[sl3 - 1] + cell[sl3 - 1]
                cell = cell.repeat(sl3,1,1)
                
        x = target[0]
        s5=0.2
        for t in range(t2-1):
            if(ct3 == "LSTM"):
                output, hidden, cell = self.decoder(x, hidden, cell,enco_output)
                s5=s5*2
            else :
                output, hidden = self.decoder(x, hidden,cell=0,enco_out=enco_output)
                s5=s5+2

            outputs[t+1] = output
            best_guess = output.argmax(1)

            x = target[t+1] if random.random() < teacher_force_ratio else best_guess

        return outputs


In [20]:
def Evaluate(model, english_matrix, telugu_matrix, epoch, batch_size,s=0.1):
    correct_count = 0
    bat = (int)(len(english_matrix) / batch_size)
    for batch_idx in range(1,bat+1):
        inp_data = english_matrix[batch_size * (batch_idx-1) : batch_size * (batch_idx)].to(device)
        id = inp_data
        bs = batch_size
        target = telugu_matrix[bs * (batch_idx-1) : bs * (batch_idx)].to(device)
        tgt = target
        output = model.forward(id.T, tgt.T, 0)
        s1=s
        output = nn.Softmax(dim=2)(output)
        #s1=s1*bs
        output = torch.argmax(output, dim=2)
        ot = output
        output = ot.T

        for i in range(1,bs+1):
            if(torch.equal(output[i-1][1:],target[i-1][1:])):
                k = correct_count
                correct_count = k+1
    l5 = len(english_matrix)
    accuracy = correct_count * 100 / l5
    return accuracy
        

In [21]:
def to_words(model, english_matrix, telugu_matrix, batch_size):
  bs=batch_size
  count=0
  for batch_idx in range((int)(len(english_matrix) / bs)):
        inp_data = english_matrix[bs * batch_idx : bs * (batch_idx+1)].to(device)
        inp_data1=inp_data
        target = telugu_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        count=count+bs
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        gt1=[]
        output = output.T
        count=count/(batch_idx+1)
        for i in range(len(inp_data)):
          inp_word = inp_data[i]
          word = ""
          for j in range(len(inp_word)):
            eng_char_array_size=27
            if(inp_word[j]>0 and inp_word[j]<eng_char_array_size):
              word += english_char_list[inp_word[j]-1]
          #temp1 = [word]
          #if(data_type == "test"):
          inp.append(word)
        for i in range(len(target)):
          word1 = ""
          word2 = ""
          target_word = target[i]
          output_word = output[i]
          gt1.append(output_word)
          for j in range(len(target_word)):
            if(target_word[j]>0 and target_word[j]<63):
              word1 += telugu_char_list[target_word[j] - 1]
          count1=count
          for j in range(len(output_word)):
            if(output_word[j]>0 and output_word[j]<63):
              word2 = word2 + telugu_char_list[output_word[j] - 1]
          temp = [word1, word2]
          
          gt.append(temp[0])
          pred.append(temp[1])
        
  df_test = pd.DataFrame({"Input": inp, "Ground Truth" : gt, "Model output":pred})
  df_test.to_csv("prediction_Attention.csv", index=False)
            

In [22]:
def neural_network(cell_type, bidirectional, num_layers, batch_size, embedding_size, hidden_size, dropout,num_epochs=5,learning_rate=0.001):
  x=len(english_char_list)
  y=len(telugu_char_list)
  input_size_encoder = x + 2   
  input_size_decoder = y + 2  
  output_size        = y + 2  
  s=0.2
  encoder_net = Encoder(input_size_encoder, embedding_size, hidden_size, num_layers, dropout, cell_type,bidirectional).to(device)
  s=s+1
  decoder_net = Decoder(input_size_decoder,embedding_size,hidden_size,output_size,num_layers,dropout, cell_type, bidirectional).to(device)
  pad_idx = 63
  model = Seq2Seq(encoder_net, decoder_net, cell_type, bidirectional, num_layers).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  
  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

  # train_loss = []
  # train_accuracy = []
  # val_accuracy = []
  for epoch in range(1,num_epochs+1):
      ep = epoch-1
      print("epoch = ",ep)

      model.train()
      step = 0
      total_loss = 0
      bat = (int)(len(english_matrix) / batch_size)
      for batch_idx in range(1,bat+1):
          
          bi = batch_idx
          inp_data = english_matrix[batch_size * (bi-1) : batch_size * (bi)].to(device)
          target = telugu_matrix[batch_size * (bi-1) : batch_size * (bi)].to(device)

          target = target.T
          output = model(inp_data.T, target)
          step1 = step
          output = output[1:].reshape(-1, output.shape[2])
          s=0
          target = target[1:].reshape(-1)
          optimizer.zero_grad()
          loss = criterion(output, target)
          l1 = loss
          total_loss = total_loss + loss
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
          optimizer.step()
          s=s+1
          step = step + 1
  #     print("total loss = ",total_loss / (len(english_matrix) * len(marathi_matrix[0])))
      tl = total_loss/step
      print("total loss = ",tl)
     
  #return train_loss,train_accuracy,val_accuracy
  test_accuracy = Evaluate(model, english_matrix_test, telugu_matrix_test, num_epochs, batch_size)
  print("Test accuracy = ",test_accuracy)
  to_words(model, english_matrix_test, telugu_matrix_test, batch_size)

In [23]:
#best hyperparameters
cell_type = "GRU"
bidirectional = True
num_layers = 1
batch_size = 128
embedding_size = 128
hidden_size = 512
dropout = 0.4
num_epochs = 15
learning_rate = 0.001
neural_network(cell_type, bidirectional, num_layers, batch_size, embedding_size, hidden_size, dropout, num_epochs, learning_rate)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


epoch =  0
total loss =  tensor(0.6032, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  1
total loss =  tensor(0.1832, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  2
total loss =  tensor(0.1425, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  3
total loss =  tensor(0.1176, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  4
total loss =  tensor(0.1065, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  5
total loss =  tensor(0.0971, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  6
total loss =  tensor(0.0837, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  7
total loss =  tensor(0.0770, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  8
total loss =  tensor(0.0713, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  9
total loss =  tensor(0.0662, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  10
total loss =  tensor(0.0600, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  11
total loss =  tensor(0.0588, device='cuda:0', grad_fn=<DivBackward0>)
epoch =  12
total loss =  

In [25]:
df_test = pd.read_csv("prediction_Attention.csv")
df_test = df_test.sample(frac=1)
df_test.head(20)

,Input,Ground Truth,Model output
223,poosi,పూసి,పూసి
221,ghatane,ఘటనే,ఘటనే
3000,saagutondani,సాగుతోందని,సాగుతోందని
1201,wek,వేక్,వెక్
1821,meeda,మేడ,మీద
1187,palisetti,పలిశెట్టి,పలిసెట్టి
3306,kshinathaku,క్షీణతకు,క్షిణతకు
2658,sandarbhamuga,సందర్భముగా,సందర్భముగా
1930,sasaram,ససరం,ససరం
3307,bollaapalli,బొల్లాపల్లి,బొల్లాపల్లి


In [ ]:
# sweep_config = {
#     "method": 'bayes', #grid, random
#     "metric": {
#       "name": 'validation_accuracy',
#       "goal": 'maximize', 
#     },
#     "parameters": {
#         "epochs": {
#             "values": [15]
#         },
#         "dropout": {
#             "values": [0.5]
#         },
#         "hidden_size": {
#             "values": [256]
#         },
#         "embedding_size": {
#             "values": [128]
#         },
#         "learning_rate":{
#             "values":[0.001]
#         },
#         "batch_size":{
#             "values":[128]
#         },
#         "num_layers":{
#             "values":[2]
#        },
#         "bidirectional":{
#             "values":[False]
#         },
#         "cell_type":{
#             "values":['LSTM']
#         }
#     }
# }

In [ ]:
# sweep_config = {
#     "method": 'bayes', #grid, random
#     "metric": {
#       "name": 'validation_accuracy',
#       "goal": 'maximize', 
#     },
#     "parameters": {
#         "epochs": {
#             "values": [15]
#         },
#         "dropout": {
#             "values": [0.3,0.4,0.5]
#         },
#         "hidden_size": {
#             "values": [128,256,512]
#         },
#         "embedding_size": {
#             "values": [128,256,512]
#         },
#         "learning_rate":{
#             "values":[0.001]
#         },
#         "batch_size":{
#             "values":[32,64,128,256]
#         },
#         "num_layers":{
#             "values":[1,3,2]
#        },
#         "bidirectional":{
#             "values":[False,True]
#         },
#         "cell_type":{
#             "values":['LSTM','RNN','GRU']
#         }
#     }
# }

In [ ]:
# def train():
   
#     wandb.init()

#     #wandb.init()

#     # Config is a variable that holds and saves hyperparameters and inputs
#     config = wandb.config

#     name="ct_"+str(wandb.config.cell_type)+"_bd_"+str(wandb.config.bidirectional)+"_nl_"+str(wandb.config.num_layers)+"_bs_"+str(wandb.config.batch_size)+"_es_"+str(wandb.config.embedding_size)+"_hs_"+str(wandb.config.hidden_size)+"_dp_"+str(wandb.config.dropout)
#     print(name)
#     wandb.run.name = name



    
  

 
#     #  Model training here
#     #sweep_network = FFNNClass(hidden_sizes=config.hidden_sizes ,algo=config.algo,initialization = config.initialization,activation=config.activation,loss=config.loss)
#     #neural_network(cell_type, bidirectional, num_layers,batch_size,embedding_size,hidden_size,dropout,epochs,learning_rate)
#     neural_network(cell_type=config.cell_type, bidirectional=config.bidirectional, num_layers=config.num_layers,batch_size = config.batch_size,embedding_size=config.embedding_size,hidden_size=config.hidden_size,dropout=config.dropout,num_epochs=config.epochs,learning_rate=config.learning_rate)

  

#     # for zi in range(len(train_acc)):
#     #     print(zi)        
#     #     wandb.log({'train_accuracy':train_acc[zi]})
#     #     wandb.log({'val_accuracy':val_acc[zi]})
#     #     wandb.log({'train_loss':train_loss[zi]}) 
#     #     #wandb.log({'val_loss':val_loss[zi]})
#     #     wandb.log({'num_epochs':zi+1})

#     wandb.finish()

In [ ]:
# sweep_id = wandb.sweep(sweep_config, entity="cs22m043", project="A3_with_Attn")
# wandb.agent(sweep_id, train, count=5)

Create sweep with ID: csfwnf6s
Sweep URL: https://wandb.ai/cs22m043/A3_with_Attn/sweeps/csfwnf6s


wandb: Agent Starting Run: 7y9xmso0 with config:
wandb: 	batch_size: 128
wandb: 	bidirectional: True
wandb: 	cell_type: GRU
wandb: 	dropout: 0.2
wandb: 	embedding_size: 64
wandb: 	epochs: 15
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 2
wandb: Currently logged in as: cs22m043. Use `wandb login --relogin` to force relogin


ct_GRU_bd_True_nl_2_bs_128_es_64_hs_256_dp_0.2
epoch =  0
total loss =  tensor(1.0974, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  9.927734375
Validation accuracy =  13.4033203125
epoch =  1
total loss =  tensor(0.3251, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  42.083984375
Validation accuracy =  39.4287109375
epoch =  2
total loss =  tensor(0.1971, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  52.083984375
Validation accuracy =  45.3857421875
epoch =  3
total loss =  tensor(0.1566, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  56.599609375
Validation accuracy =  47.0947265625
epoch =  4
total loss =  tensor(0.1357, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  61.681640625
Validation accuracy =  50.6103515625
epoch =  5
total loss =  tensor(0.1221, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  65.443359375
Validation accuracy =  52.1240234375
epoch =  6
total loss =  tensor(0.1085, de

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▄▅▆▆▇▇▇▇█▇████
training_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▅▆▇▇▇▇████████
epoch,14
train_accuracy,77.82227
training_loss,0.05813
validation_accuracy,54.56543


wandb: Agent Starting Run: vbj20kqz with config:
wandb: 	batch_size: 32
wandb: 	bidirectional: False
wandb: 	cell_type: RNN
wandb: 	dropout: 0.4
wandb: 	embedding_size: 16
wandb: 	epochs: 15
wandb: 	hidden_size: 16
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 3


...

ct_LSTM_bd_True_nl_2_bs_128_es_256_hs_32_dp_0.5
epoch =  0
total loss =  tensor(1.8353, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.0
Validation accuracy =  0.0
epoch =  1
total loss =  tensor(1.3782, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.0
Validation accuracy =  0.0
epoch =  2
total loss =  tensor(1.2611, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.017578125
Validation accuracy =  0.1220703125
epoch =  3
total loss =  tensor(1.1356, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.06640625
Validation accuracy =  0.439453125
epoch =  4
total loss =  tensor(1.0613, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.166015625
Validation accuracy =  0.634765625
epoch =  5
total loss =  tensor(1.0014, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.3671875
Validation accuracy =  1.0009765625
epoch =  6
total loss =  tensor(0.9523, device='cuda:0', grad_fn=<DivBackward0>)
Training A

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▁▁▁▁▂▂▂▃▄▄▅▆▇█
training_loss,█▅▅▄▃▃▃▂▂▂▂▂▁▁▁
validation_accuracy,▁▁▁▁▁▂▂▃▃▅▅▆▆▇█
epoch,14
train_accuracy,4.41992
training_loss,0.66731
validation_accuracy,9.30176


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 5g8ep7q4 with config:
wandb: 	batch_size: 128
wandb: 	bidirectional: True
wandb: 	cell_type: GRU
wandb: 	dropout: 0.2
wandb: 	embedding_size: 32
wandb: 	epochs: 15
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 3


ct_GRU_bd_True_nl_3_bs_128_es_32_hs_128_dp_0.2
epoch =  0
total loss =  tensor(1.6151, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.001953125
Validation accuracy =  0.0244140625
epoch =  1
total loss =  tensor(1.1180, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.458984375
Validation accuracy =  1.26953125
epoch =  2
total loss =  tensor(0.8255, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  3.326171875
Validation accuracy =  6.5185546875
epoch =  3
total loss =  tensor(0.6594, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  6.7109375
Validation accuracy =  11.5234375
epoch =  4
total loss =  tensor(0.5463, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  12.244140625
Validation accuracy =  17.138671875
epoch =  5
total loss =  tensor(0.4616, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  17.12890625
Validation accuracy =  21.38671875
epoch =  6
total loss =  tensor(0.3970, device='cuda:0', gr

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▁▁▂▃▃▄▅▅▆▇▇▇██
training_loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁
validation_accuracy,▁▁▂▃▄▄▅▆▆▇▇▇███
epoch,14
train_accuracy,54.66602
training_loss,0.15392
validation_accuracy,46.41113
